In [1]:
!pip install langchain_community
!pip install chromadb
!pip install tiktoken
from IPython import get_ipython
from IPython.display import display
from IPython import get_ipython
from IPython.display import display
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import json
import chromadb
import os

os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [2]:

def generate_real_estate_listings(num_listings=10):
    """Generates realistic real estate listings in Singapore using OpenAI."""
    # Initialize the OpenAI object from langchain
    llm = OpenAI(temperature=0.7)  # Adjust temperature as needed

    listings = []
    for _ in range(num_listings):
        prompt = """Generate a diverse and realistic real estate listing in Singapore.
        Include details like address (or district), property type (condo, HDB, landed),
        number of bedrooms and bathrooms, size in square feet, key features (e.g., swimming pool, balcony,
        modern kitchen), nearby amenities (schools, MRT stations, shopping malls), price,
        and any unique selling points.  Ensure the address or district is plausible in Singapore.
        Use specific names for schools and malls that actually exist in Singapore.
        Output as a JSON object.
        """

        # Call the llm to generate the listing using the prompt
        listing = llm(prompt)
        listings.append(listing)

    return listings

In [3]:
# Example usage
listings = generate_real_estate_listings()
for listing in listings:
    print(listing)
    print("-" * 20)

<ipython-input-2-d16a77f700aa>:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.7)  # Adjust temperature as needed
<ipython-input-2-d16a77f700aa>:18: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  listing = llm(prompt)



{
    "address": "Marina Bay Sands Residences, Marina Bay",
    "property_type": "Condo",
    "bedrooms": 3,
    "bathrooms": 2,
    "size_sqft": 1700,
    "key_features": ["Infinity pool", "Sky garden", "Gymnasium"],
    "nearby_amenities": ["Marina Bay MRT Station", "Marina Bay Sands Shopping Mall", "Singapore Management University"],
    "price": "$3,500,000",
    "unique_selling_points": "Luxurious living in the heart of Singapore's central business district with stunning views of the iconic Marina Bay Sands. This fully furnished 3 bedroom, 2 bathroom condo boasts modern and stylish interiors, a spacious balcony, and access to world-class facilities such as the infinity pool and sky garden. With top schools like Singapore Management University within walking distance and easy access to public transportation and shopping at Marina Bay Sands, this is the perfect home for professionals and families looking for convenience and luxury.",
}
--------------------

{
    "address": "Marina

In [4]:
try:
    #Attempt to parse as JSON
    listing_json = json.loads(listing)
    #Use the JSON data
    print(listing_json["address"]) #Access specific fields
except json.JSONDecodeError:
    print("Could not parse as JSON, here is the raw output:")
listing

Marina Bay Sands, Marina Bay


'\n{\n    "address": "Marina Bay Sands, Marina Bay",\n    "property_type": "Condo",\n    "bedrooms": 3,\n    "bathrooms": 2,\n    "size": "1,500 sqft",\n    "key_features": ["Infinity pool", "Balcony with panoramic view", "Fully equipped modern kitchen"],\n    "nearby_amenities": {\n        "schools": ["Singapore Management University", "Cantonment Primary School", "River Valley Primary School"],\n        "mrt_stations": ["Marina Bay MRT", "Bayfront MRT"],\n        "shopping_malls": ["Marina Bay Sands Shoppes", "Marina Square", "The Shoppes at Marina Bay Sands"]\n    },\n    "price": "$4,500,000",\n    "unique_selling_points": ["Iconic waterfront location", "Luxurious lifestyle with access to Marina Bay Sands amenities", "Close proximity to top schools and MRT stations"]\n}'

In [10]:
def generate_real_estate_listings(num_listings=10):
    """Generates realistic real estate listings in Singapore using OpenAI."""
    # Initialize the OpenAI object from langchain
    llm = OpenAI(temperature=0.7)  # Adjust temperature as needed

    listings = []
    for _ in range(num_listings):
        prompt = """Generate a diverse and realistic real estate listing in Singapore.
        Include details like address (or district), property type (condo, HDB, landed),
        number of bedrooms and bathrooms, size in square feet, key features (e.g., swimming pool, balcony,
        modern kitchen), nearby amenities (schools, MRT stations, shopping malls), price,
        and any unique selling points.  Ensure the address or district is plausible in Singapore.
        Use specific names for schools and malls that actually exist in Singapore.
        Output as a JSON object.
        """

        # Call the llm to generate the listing using the prompt
        listing = llm(prompt)
        listings.append(listing)

    return listings

# Example usage
listings = generate_real_estate_listings(20) # Reduced number of listings for demonstration


In [11]:
# Initialize ChromaDB
client = chromadb.Client()

# Instead of creating, try to get the collection, if it doesn't exist, create it
try:
    collection = client.get_collection("real_estate_listings")
except:
    collection = client.create_collection("real_estate_listings")


embeddings_model = OpenAIEmbeddings()

for listing in listings:
    try:
        listing_json = json.loads(listing)
        listing_text = json.dumps(listing_json)  # Convert JSON object back to string
        embedding = embeddings_model.embed_documents([listing_text])

        # Convert list and dict metadata values to strings
        for key, value in listing_json.items():
            if isinstance(value, (list, dict)): # Check if value is a list OR a dict
                listing_json[key] = ", ".join(map(str, value))  # Join list/dict items with a comma after converting them to string

        collection.add(
            documents=[listing_text],
            embeddings=embedding,
            metadatas=[listing_json],  # Store metadata for easy retrieval
            ids=[str(listing_json.get("id", len(collection.get()['ids'])))]  # Add a unique ID for each listing
        )
    except json.JSONDecodeError:
        print(f"Could not parse listing as JSON:\n{listing}")
        continue

# Example query (replace with actual search query)
query_text = "luxury condo near Orchard Road"
query_embedding = embeddings_model.embed_query(query_text)
results = collection.query(query_embeddings=query_embedding, n_results=2)

results


Could not parse listing as JSON:

{
  "address": "8 Sentosa Gateway, Sentosa Cove, Singapore 098269",
  "property_type": "condo",
  "bedrooms": "4",
  "bathrooms": "4",
  "size": "2,500 sq ft",
  "key_features": ["private swimming pool", "spacious balcony", "modern kitchen"],
  "nearby_amenities": ["Sentosa Golf Club", "Sentosa Cove Village", "Quayside Isle", "Sentosa Cove MRT Station"],
  "price": "$7.5 million",
  "unique_selling_points": "This stunning 4-bedroom condo is located in the prestigious Sentosa Cove area, offering a luxurious and exclusive lifestyle. The unit boasts a private swimming pool, perfect for relaxing and entertaining, as well as a spacious balcony with breathtaking views of the marina. The modern kitchen is equipped with high-end appliances and finishes. Residents can enjoy easy access to nearby amenities, such as the Sentosa Golf Club, Sentosa Cove Village, and Quayside Isle for dining and shopping. The Sentosa Cove MRT Station is also just a short walk away, 

{'ids': [['1', '11']],
 'embeddings': None,
 'documents': [['{"address": "Orchard Road, District 9", "property_type": "Condo", "bedrooms": 3, "bathrooms": 2, "size_sqft": 1500, "key_features": ["Swimming Pool", "Balcony", "Modern Kitchen"], "nearby_amenities": ["Anglo-Chinese School (Junior)", "Newton MRT Station", "ION Orchard"], "price": "$2,500,000", "unique_selling_points": "Prime location in the heart of Orchard Road with easy access to prestigious schools, public transportation, and world-renowned shopping malls."}',
   '{"address": "8 Amber Gardens, District 15", "property_type": "Condo", "bedrooms": 3, "bathrooms": 2, "size": 1200, "key_features": ["Swimming Pool", "Balcony", "Modern Kitchen"], "nearby_amenities": ["Tanjong Katong Primary School", "Katong Shopping Centre", "Dakota MRT Station"], "price": "$1,500,000", "unique_selling_points": "Located in the exclusive Amber Gardens area, this stunning condo offers luxurious living with its spacious bedrooms and modern kitchen. 

In [12]:

def semantic_search(query_text, n_results=2):
    """
    Performs a semantic search on the real estate listings.
    """
    embeddings_model = OpenAIEmbeddings()
    query_embedding = embeddings_model.embed_query(query_text)

    # Use Chroma's query method
    results = collection.query(query_embeddings=query_embedding, n_results=n_results)

    return results

# Example Usage
query_text = "luxury condo near District 10 with 3 bedrooms and a swimming pool"
results = semantic_search(query_text, n_results=3)

# Process and display the results
print(f"Search query: {query_text}")
if results and results['documents']:
    for i, doc in enumerate(results['documents'][0]):
        print(f"\nResult {i + 1}:")
        try:
          listing = json.loads(doc)
          print(json.dumps(listing, indent=2))
        except json.JSONDecodeError:
          print("Could not parse the document as JSON:", doc)
else:
    print("No matching listings found.")

Search query: luxury condo near District 10 with 3 bedrooms and a swimming pool

Result 1:
{
  "address": "8 Amber Gardens, District 15",
  "property_type": "Condo",
  "bedrooms": 3,
  "bathrooms": 2,
  "size": 1200,
  "key_features": [
    "Swimming Pool",
    "Balcony",
    "Modern Kitchen"
  ],
  "nearby_amenities": [
    "Tanjong Katong Primary School",
    "Katong Shopping Centre",
    "Dakota MRT Station"
  ],
  "price": "$1,500,000",
  "unique_selling_points": "Located in the exclusive Amber Gardens area, this stunning condo offers luxurious living with its spacious bedrooms and modern kitchen. The highlight of this property is its large swimming pool and balcony, perfect for relaxing and entertaining. It is also conveniently located within walking distance to Tanjong Katong Primary School, ideal for families with young children. The nearby Katong Shopping Centre offers a variety of dining and shopping options for residents. With easy access to the Dakota MRT Station, commuting 

In [9]:
def augment_listing_description(listing, buyer_preferences):
    """Augments a real estate listing description based on buyer preferences."""

    llm = OpenAI(temperature=0.7)

    prompt = f"""
    Rewrite the following real estate listing to highlight features appealing to a buyer with the following preferences: {buyer_preferences}.
    Do not change any factual information about the property.  Only rephrase the description to emphasize aspects that would interest this buyer.

    Listing: {json.dumps(listing)}
    """
    augmented_description = llm(prompt)
    return augmented_description

# Example usage (assuming 'results' from the semantic search is available):
if results and results['documents']:
    for i, doc in enumerate(results['documents'][0]):
        try:
            listing = json.loads(doc)
            buyer_preferences = "young professional, enjoys nightlife, and is looking for a modern condo with easy access to public transport"  # Example preferences
            augmented_listing = augment_listing_description(listing, buyer_preferences)
            print(f"\nAugmented Result {i+1}:")
            print(augmented_listing)

        except json.JSONDecodeError:
            print("Could not parse the document as JSON:", doc)
else:
    print("No matching listings found.")


Augmented Result 1:

This modern condo on Orchard Road is the perfect home for a young professional looking for a vibrant lifestyle. With its prime location in District 9, you'll have easy access to the city's best nightlife, including popular spots like ION Orchard. Located near Newton MRT Station, you'll have convenient access to public transport, making it easy to get around the city. Plus, with 3 bedrooms and 2 bathrooms, this spacious condo offers plenty of room for entertaining friends. The condo also boasts a swimming pool, balcony, and modern kitchen, making it the perfect space to relax and unwind after a long day. Don't miss out on this opportunity to live in the heart of the action! 

Augmented Result 2:

"Live in the heart of Singapore's nightlife and modern city living with this stunning 3 bedroom condo located at 12 Marina Boulevard in Marina Bay. Enjoy easy access to public transport with the nearby Marina Bay MRT Station and be just steps away from the bustling Marina 